<a href="https://colab.research.google.com/github/skashyapsri/ghi-forecasting/blob/main/adversarial_sparse_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
import requests
from datetime import datetime


def process_nasa_data(data):
    ghi_data = []
    time_data = []

    ghi_dict = data['properties']['parameter']['ALLSKY_SFC_SW_DWN']
    for timestamp, value in ghi_dict.items():
        if len(timestamp) == 10:  # Full timestamp format YYYYMMDDHH
            ghi_data.append(float(value))
            time_data.append(datetime.strptime(timestamp, '%Y%m%d%H'))

    return np.array(ghi_data)


# NASA POWER API data fetcher
def fetch_power_data(lat, lon, start_date, end_date):
    base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"
    params = {
        "parameters": "ALLSKY_SFC_SW_DWN",  # GHI parameter
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Data preprocessing
class DataPreprocessor:
    def __init__(self, window_size=24, prediction_hours=1):
        self.window_size = window_size
        self.prediction_hours = prediction_hours
        self.scaler = StandardScaler()

    def create_sequences(self, data):
        X, y = [], []
        data = data.reshape(-1, 1)
        for i in range(len(data) - self.window_size - self.prediction_hours):
            X.append(data[i:(i + self.window_size)])
            y.append(data[i + self.window_size:i + self.window_size + self.prediction_hours])
        return np.array(X), np.array(y)

    def normalize_data(self, data):
        data = data.reshape(-1, 1)
        return self.scaler.fit_transform(data)

# AST Model Implementation
class AST(keras.Model):
    def __init__(self, seq_len, pred_len, d_model=256, n_heads=4, n_layers=3):
        super(AST, self).__init__()

        self.seq_len = seq_len
        self.pred_len = pred_len

        # Embedding layers
        self.input_proj = layers.Dense(d_model)
        self.pos_encoding = self._positional_encoding(seq_len, d_model)

        # Sparse Transformer Encoder
        self.encoder_layers = [
            SparseTransformerLayer(d_model, n_heads)
            for _ in range(n_layers)
        ]

        # Output projection
        self.output_proj = layers.Dense(1)

    def _positional_encoding(self, seq_len, d_model):
        positions = np.arange(seq_len)[:, np.newaxis]
        angles = np.arange(d_model)[np.newaxis, :] / d_model
        angles = positions * angles

        angles[:, 0::2] = np.sin(angles[:, 0::2])
        angles[:, 1::2] = np.cos(angles[:, 1::2])

        pos_encoding = angles[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, inputs, training=False):
        # Input projection and positional encoding
        x = self.input_proj(inputs)
        x += self.pos_encoding

        # Encoder layers
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)

        # Output projection
        output = self.output_proj(x)
        return output

class SparseTransformerLayer(layers.Layer):
    def __init__(self, d_model, n_heads):
        super(SparseTransformerLayer, self).__init__()

        self.mha = layers.MultiHeadAttention(n_heads, d_model)
        self.ffn = keras.Sequential([
            layers.Dense(d_model * 4, activation='relu'),
            layers.Dense(d_model)
        ])

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)

    def call(self, inputs, training=False):
        # Multi-head attention with sparse attention mask
        attn_output = self.mha(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)

        # Feed forward network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Training
def train_model(model, train_data, val_data, epochs=100):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss_fn = tf.keras.losses.MeanSquaredError()

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            predictions = model(x, training=True)
            loss = loss_fn(y, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss

    @tf.function
    def val_step(x, y):
        predictions = model(x, training=False)
        loss = loss_fn(y, predictions)
        return loss

    for epoch in range(epochs):
        # Track loss for each epoch
        train_losses = []
        val_losses = []

        for x_batch, y_batch in train_data:
            train_loss = train_step(x_batch, y_batch)
            train_losses.append(float(train_loss))

        for x_batch, y_batch in val_data:
            val_loss = val_step(x_batch, y_batch)
            val_losses.append(float(val_loss))

        print(f"Epoch {epoch + 1}, "
              f"Train Loss: {np.mean(train_losses):.4f}, "
              f"Val Loss: {np.mean(val_losses):.4f}")

# Main execution
def main():
    # Fetch data
    lat, lon = 40.7128, -74.0060  # New York City coordinates
    data = fetch_power_data(lat, lon, "20230101", "20231231")
    ghi_data = process_nasa_data(data)

    # Preprocess
    preprocessor = DataPreprocessor(window_size=24, prediction_hours=1)
    normalized_data = preprocessor.normalize_data(ghi_data)
    X, y = preprocessor.create_sequences(normalized_data)

    # Split data
    split = int(0.8 * len(X))
    X_train, X_val = X[:split], X[split:]
    y_train, y_val = y[:split], y[split:]

    # Create data loaders
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32)

    # Initialize and train model
    model = AST(seq_len=24, pred_len=1)
    train_model(model, train_dataset, val_dataset)

    # Save model
    model.save('ast_ghi_model.h5')

In [ ]:
import os
CUDA_LAUNCH_BLOCKING=1
TORCH_USE_CUDA_DSA=1
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"
os.environ["PYTORCH_USE_CUDA_DSA"] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import requests
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import math

# NASA POWER API data fetcher
def fetch_power_data(lat, lon, start_date, end_date):
    base_url = "https://power.larc.nasa.gov/api/temporal/hourly/point"
    params = {
        "parameters": "ALLSKY_SFC_SW_DWN",  # GHI parameter
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": start_date,
        "end": end_date,
        "format": "JSON"
    }
    response = requests.get(base_url, params=params)
    return response.json()

class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length, pred_length):
        self.data = torch.FloatTensor(data)
        self.seq_length = seq_length
        self.pred_length = pred_length

    def __len__(self):
        return len(self.data) - self.seq_length - self.pred_length + 1

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.seq_length].unsqueeze(-1)  # Add feature dimension
        y = self.data[idx + self.seq_length:idx + self.seq_length + self.pred_length].unsqueeze(-1)
        return x, y

class AlphaEntmax(nn.Module):
    def __init__(self, alpha=1.5, dim=-1):
        super().__init__()
        self.alpha = alpha
        self.dim = dim

    def forward(self, x):
        return self.entmax(x, self.alpha, dim=self.dim)

    def entmax(self, x, alpha, dim=-1):
        if alpha == 1:
            return torch.softmax(x, dim=dim)

        x_shifted = x - x.max(dim=dim, keepdim=True)[0]
        tau = self._find_tau(x_shifted, alpha, dim)
        p = torch.clamp(((alpha - 1) * x_shifted - tau) / alpha, min=0) ** (1 / (alpha - 1))
        return p

    def _find_tau(self, x, alpha, dim):
        n = x.shape[dim]
        tau_lower = x.min(dim=dim, keepdim=True)[0] * (alpha - 1)
        tau_upper = x.max(dim=dim, keepdim=True)[0] * (alpha - 1)

        for _ in range(20):
            tau = (tau_lower + tau_upper) / 2
            p = torch.clamp(((alpha - 1) * x - tau) / alpha, min=0) ** (1 / (alpha - 1))
            sum_p = p.sum(dim=dim, keepdim=True)
            too_high = (sum_p > 1)
            tau_lower = torch.where(too_high, tau, tau_lower)
            tau_upper = torch.where(too_high, tau_upper, tau)

        return tau

class SparseMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1, alpha=1.5):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

        self.alpha_entmax = AlphaEntmax(alpha=alpha)
        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        # Linear transformations
        q = self.q_linear(q).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(k).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(v).view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Compute attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if mask is not None:
            # Ensure mask matches scores dimensions
            if mask.dim() == 3:
                mask = mask.unsqueeze(1)
            elif mask.dim() == 2:
                mask = mask.unsqueeze(1).unsqueeze(2)

            # Ensure mask size matches scores size
            scores_size = scores.size()
            mask = mask[:, :, :scores_size[2], :scores_size[3]]

            scores = scores.masked_fill(mask == 0, float('-inf'))

        # Apply sparse attention
        attn = self.alpha_entmax(scores)
        attn = self.dropout(attn)

        # Compute weighted sum
        output = torch.matmul(attn, v)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        return self.out_linear(output)

def create_mask(seq_len):
    """
    Creates a causal mask for the decoder self-attention.
    Args:
        seq_len: Length of the sequence
    Returns:
        mask: Binary mask where 1 indicates attention is allowed
    """
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
    return ~mask  # Invert to get attention mask where 1 means attend, 0 means don't attend

def create_src_mask(src):
    """
    Creates a mask for the encoder self-attention.
    Args:
        src: Source sequence tensor of shape [batch_size, seq_len, ...]
    Returns:
        mask: Binary mask where 1 indicates attention is allowed
    """
    batch_size, seq_len = src.size(0), src.size(1)
    return torch.ones((batch_size, seq_len, seq_len)).to(src.device)

def create_tgt_mask(tgt):
    """
    Creates a causal mask for the decoder self-attention.
    Args:
        tgt: Target sequence tensor of shape [batch_size, seq_len, ...]
    Returns:
        mask: Binary mask where 1 indicates attention is allowed
    """
    batch_size, seq_len = tgt.size(0), tgt.size(1)
    mask = create_mask(seq_len)
    return mask.unsqueeze(0).expand(batch_size, -1, -1).to(tgt.device)

def train_ast(data, seq_length=168, pred_length=24, batch_size=32, epochs=100,
              d_model=256, num_heads=8, num_layers=3, dropout=0.1):

    # Create dataset and dataloader
    dataset = TimeSeriesDataset(data, seq_length, pred_length)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize models
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AST(input_dim=1, output_dim=1, d_model=d_model,
               num_heads=num_heads, num_layers=num_layers, dropout=dropout).to(device)
    discriminator = Discriminator(pred_length).to(device)

    # Initialize optimizers
    g_optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0001)

    # Loss functions
    mse_loss = nn.MSELoss()
    bce_loss = nn.BCELoss()

    print("Training on device:", device)

    for epoch in range(epochs):
        model.train()
        discriminator.train()
        total_g_loss = 0
        total_d_loss = 0

        for batch_idx, (x, y) in enumerate(dataloader):
            batch_size = x.size(0)
            x, y = x.to(device), y.to(device)

            # Train discriminator
            d_optimizer.zero_grad()

            # Generate fake sequence
            with torch.no_grad():
                fake_seq = model(x, y)

            # Real and fake labels
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)

            # Discriminator predictions
            d_real = discriminator(y)
            d_fake = discriminator(fake_seq.detach())

            # Discriminator loss
            d_real_loss = bce_loss(d_real, real_labels)
            d_fake_loss = bce_loss(d_fake, fake_labels)
            d_loss = d_real_loss + d_fake_loss

            d_loss.backward()
            d_optimizer.step()

            # Train generator
            g_optimizer.zero_grad()

            fake_seq = model(x, y)
            d_fake = discriminator(fake_seq)

            # Generator loss (combine MSE and adversarial loss)
            mse = mse_loss(fake_seq, y)
            adversarial_loss = bce_loss(d_fake, real_labels)
            g_loss = mse + 0.1 * adversarial_loss

            g_loss.backward()
            g_optimizer.step()

            total_g_loss += g_loss.item()
            total_d_loss += d_loss.item()

        if (epoch + 1) % 10 == 0:
            avg_g_loss = total_g_loss / len(dataloader)
            avg_d_loss = total_d_loss / len(dataloader)
            print(f'Epoch [{epoch+1}/{epochs}], G Loss: {avg_g_loss:.4f}, D Loss: {avg_d_loss:.4f}')

    return model, discriminator

class AST(nn.Module):
    def __init__(self, input_dim=1, output_dim=1, d_model=256, num_heads=8, num_layers=3, dropout=0.1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.d_model = d_model

        # Input embedding
        self.input_embedding = nn.Linear(input_dim, d_model)

        # Positional encoding
        self.register_buffer('pos_encoding', self._create_positional_encoding(5000, d_model))

        # Encoder and decoder layers with layer normalization
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])

        self.decoder_layers = nn.ModuleList([
            nn.TransformerDecoderLayer(d_model=d_model, nhead=num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])

        # Output projection
        self.output_linear = nn.Linear(d_model, output_dim)

    def _create_positional_encoding(self, max_seq_len, d_model):
        pos_encoding = torch.zeros(max_seq_len, d_model)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding

    def forward(self, src, tgt):
        # Add batch dimension if not present
        if src.dim() == 2:
            src = src.unsqueeze(0)
        if tgt.dim() == 2:
            tgt = tgt.unsqueeze(0)

        # Create masks
        src_mask = None  # Allow attending to all source positions
        tgt_mask = self._generate_square_subsequent_mask(tgt.size(1)).to(tgt.device)

        # Embedding and positional encoding
        src = self.input_embedding(src)
        tgt = self.input_embedding(tgt)

        src = src + self.pos_encoding[:src.size(1)]
        tgt = tgt + self.pos_encoding[:tgt.size(1)]

        # Transpose for transformer input [seq_len, batch, features]
        src = src.transpose(0, 1)
        tgt = tgt.transpose(0, 1)

        # Encoder
        for enc_layer in self.encoder_layers:
            src = enc_layer(src, src_mask)

        # Decoder
        for dec_layer in self.decoder_layers:
            tgt = dec_layer(tgt, src, tgt_mask)

        # Transpose back [batch, seq_len, features]
        output = tgt.transpose(0, 1)

        return self.output_linear(output)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

class Discriminator(nn.Module):
    def __init__(self, seq_len, hidden_dim=256):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(seq_len, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1)  # Flatten the sequence
        return self.model(x)

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.self_attn = SparseMultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super().__init__()
        self.self_attn = SparseMultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attn = SparseMultiHeadAttention(d_model, num_heads, dropout)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))

        cross_attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(cross_attn_output))

        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [ ]:
lat, lon = 40.7128, -74.0060  # New York City coordinates
data = fetch_power_data(lat, lon, "20230101", "20231231")
print(data)
values = list(data['properties']['parameter']['ALLSKY_SFC_SW_DWN'].values())
values = [float(x) for x in values if isinstance(x, (int, float))]
normalized_data = (values - np.mean(values)) / np.std(values)

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-74.006, 40.7128, 10.17]}, 'properties': {'parameter': {'ALLSKY_SFC_SW_DWN': {'2023010100': 0.0, '2023010101': 0.0, '2023010102': 0.0, '2023010103': 0.0, '2023010104': 0.0, '2023010105': 0.0, '2023010106': 0.0, '2023010107': 13.75, '2023010108': 102.52, '2023010109': 247.45, '2023010110': 316.64, '2023010111': 449.04, '2023010112': 445.98, '2023010113': 377.05, '2023010114': 264.79, '2023010115': 124.88, '2023010116': 14.83, '2023010117': 0.0, '2023010118': 0.0, '2023010119': 0.0, '2023010120': 0.0, '2023010121': 0.0, '2023010122': 0.0, '2023010123': 0.0, '2023010200': 0.0, '2023010201': 0.0, '2023010202': 0.0, '2023010203': 0.0, '2023010204': 0.0, '2023010205': 0.0, '2023010206': 0.0, '2023010207': 8.92, '2023010208': 83.25, '2023010209': 189.54, '2023010210': 272.9, '2023010211': 334.23, '2023010212': 263.11, '2023010213': 264.22, '2023010214': 111.78, '2023010215': 62.09, '2023010216': 11.94, '2023010217': 0.0, '20230

In [ ]:
# Set hyperparameters
seq_length = 168  # 7 days of hourly data
pred_length = 24  # Predict next 24 hours
batch_size = 32
epochs = 100

# Train the model
model, discriminator = train_ast(
    normalized_data,
    seq_length=seq_length,
    pred_length=pred_length,
    batch_size=batch_size,
    epochs=epochs
)

Training on device: cuda
Epoch [10/100], G Loss: 0.0703, D Loss: 1.3868
Epoch [20/100], G Loss: 0.0697, D Loss: 1.3874
Epoch [30/100], G Loss: 0.0696, D Loss: 1.3869
Epoch [40/100], G Loss: 0.0695, D Loss: 1.3870
Epoch [50/100], G Loss: 0.0694, D Loss: 1.3870
Epoch [60/100], G Loss: 0.0694, D Loss: 1.3867
Epoch [70/100], G Loss: 0.0693, D Loss: 1.3868
Epoch [80/100], G Loss: 0.0694, D Loss: 1.3867
Epoch [90/100], G Loss: 0.0693, D Loss: 1.3865
Epoch [100/100], G Loss: 0.0694, D Loss: 1.3863


In [ ]:
def calculate_metrics(y_true, y_pred):
    """Calculate standard performance metrics for solar forecasting."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # R2 Score
    r2 = 1 - np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2)

    # RMSE
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))

    # rRMSE (Relative RMSE)
    rrmse = rmse / np.mean(y_true) * 100

    # MAE
    mae = np.mean(np.abs(y_true - y_pred))

    # MAPE
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Forecast Skill (FS) using persistence model as reference
    y_pers = y_true[:-1]  # Persistence forecast
    rmse_pers = np.sqrt(np.mean((y_true[1:] - y_pers)**2))
    fs = (1 - rmse/rmse_pers) * 100

    return {
        'R2': r2,
        'RMSE': rmse,
        'rRMSE': rrmse,
        'MAE': mae,
        'MAPE': mape,
        'FS': fs
    }

def denormalize(normalized_values, mean, std):
    """Convert normalized values back to original scale."""
    return normalized_values * std + mean

def test_model(model, test_data, seq_length, pred_length, data_mean, data_std):
    """Test the model and calculate performance metrics."""
    model.eval()
    device = next(model.parameters()).device

    # Create test dataset
    test_dataset = TimeSeriesDataset(test_data, seq_length, pred_length)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    all_preds = []
    all_true = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            # Generate prediction
            pred = model(x, y)

            # Denormalize predictions and true values
            pred = denormalize(pred.cpu().numpy().reshape(-1), data_mean, data_std)
            y = denormalize(y.cpu().numpy().reshape(-1), data_mean, data_std)

            all_preds.extend(pred)
            all_true.extend(y)

    # Calculate metrics
    metrics = calculate_metrics(all_true, all_preds)

    return metrics, all_preds, all_true

def evaluate_persistence_model(data, pred_length):
    """Evaluate persistence model for comparison."""
    y_true = data[pred_length:]
    y_pred = data[:-pred_length]

    return calculate_metrics(y_true, y_pred)

In [ ]:
# Evaluate persistence model
print("\nPersistence Model Performance:")
pers_metrics = evaluate_persistence_model(values[train_size:], pred_length)
for metric, value in pers_metrics.items():
  print(f"{metric}: {value:.4f}")

# Evaluate AST model
print("\nAST Model Performance:")
ast_metrics, predictions, true_values = test_model(
  model,
  test_data,
  seq_length,
  pred_length,
  data_mean,
  data_std
)
for metric, value in ast_metrics.items():
  print(f"{metric}: {value:.4f}")

# Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
plt.plot(true_values[:100], label='Actual', alpha=0.7)
plt.plot(predictions[:100], label='Predicted', alpha=0.7)
plt.title('AST Model: Actual vs Predicted GHI (First 100 hours)')
plt.xlabel('Hours')
plt.ylabel('GHI (W/m²)')
plt.legend()
plt.grid(True)
plt.show()

# Plot scatter plot
plt.figure(figsize=(8, 8))
plt.scatter(true_values, predictions, alpha=0.5)
plt.plot([min(true_values), max(true_values)],
        [min(true_values), max(true_values)],
        'r--', label='Perfect Prediction')
plt.xlabel('Actual GHI (W/m²)')
plt.ylabel('Predicted GHI (W/m²)')
plt.title('AST Model: Predicted vs Actual GHI')
plt.legend()
plt.grid(True)
plt.show()

# Print comparison with benchmark papers
print("\nComparison with Benchmark Papers:")
print("Metric    | Current Model | Best in Literature")
print("-" * 50)
print(f"RMSE      | {ast_metrics['RMSE']:.2f} W/m²    | 93.8 W/m² (1h) - 129.1 W/m² (3h)")
print(f"R²        | {ast_metrics['R2']:.3f}        | 0.88 - 0.93")
print(f"rRMSE     | {ast_metrics['rRMSE']:.2f}%       | 15.0% (1h) - 24.2% (3h)")
print(f"MAE       | {ast_metrics['MAE']:.2f} W/m²    | 30 - 100 W/m²")
print(f"MAPE      | {ast_metrics['MAPE']:.2f}%       | 10 - 25%")
print(f"FS        | {ast_metrics['FS']:.2f}%       | 20 - 40%")